In [ ]:
# xmlrpc_server.ipynb

from xmlrpc.server import SimpleXMLRPCServer
from xmlrpc.server import SimpleXMLRPCRequestHandler
import xmlrpc.client
import datetime
import pandas as pd
import pickle

class RequestHandler(SimpleXMLRPCRequestHandler):
    rpc_paths = ('/RPC2',)

server = SimpleXMLRPCServer(("localhost", 8008), 
                            requestHandler=RequestHandler)

#server.register_introspection_functions()

stats_server = xmlrpc.client.ServerProxy("http://localhost:8018")

def get_event_logs(operation_type, start_time, end_time):
    stat = stats_server.get_event_log(operation_type, start_time, end_time)
    return stat

server.register_function(get_event_logs, 'get_event_logs')

# Добавление в лог через сервер
def add_log(log_line):
    try:
        stats_server.add_log(log_line)
        return True

# Тест
def ping():
    add_log('ping')
    return True
server.register_function(ping, 'ping')

# Время сервера
def now():
    return datetime.datetime.now()
server.register_function(now, 'now')

# Отображение строкового вида, типа и значений
def show_type(arg):
    add_log('type')
    return (str(arg), str(type(arg)), arg)
server.register_function(show_type, 'type')

# Сумма
def test_sum(a, b):
    add_log('sum')
    return a + b
server.register_function(test_sum, 'sum')

# Степень
def test_pow(a, b):
    add_log('pow')
    return a**b
server.register_function(test_pow, 'pow')

# Проверка нахождения клиента в черном списке c использованием Pandas Data Frame
def black_list_check(sname):
    add_log('black_list_check')
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    exist = any(frame['Surname'] == sname)
    if (exist == True):
        return sname + ": "+ "bad_boy"
    else:
        return sname + ": "+ "good_boy"

server.register_function(black_list_check, 'black_list_check')

# Проверка нахождения клиента в черном списке c использованием Pandas Data Frame
def black_list_check_par(sname, name, patronym, date):
    add_log('black_list_check_par')
    frame = pd.read_csv('bad_boys2.csv', header=0, sep=',', encoding='utf8')
    exist = any(frame['Surname'] == sname)
    existn = any(frame['Name'] == name)
    existp = any(frame['Patronym'] == patronym)
    existd = any(frame['Birth'] == date)
    if ((exist == True) and (existn == True) and (existp == True) and (existd == True)):
        return sname + " " + name + " " + patronym + " " + date + ": "+ "bad_boy"
    else:
        return sname + " " + name + " " + patronym + " " + date +  ": "+ "good_boy"

server.register_function(black_list_check_par, 'black_list_check_par')

# Бинарная передача данных
def send_back_binary(bin_data):
    add_log('send_back_binary')
    data = bin_data.data
    return xmlrpc.client.Binary(data)
server.register_function(send_back_binary, 'send_back_binary')

# бинаризация с параметром
def send_back_binary_par(bin_data, threshold):
    add_log('send_back_binary_par')
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]
    
    if len(img_arr.shape) == 3:
        for i in range(height):
            for j in range(width):
                res = 1
                for k in range(3):
                    res *= (img_arr[i][j][k] > threshold[k]) * 255
                    
                res = 255 if res >= 255 else 0
                
                for k in range(3):
                    img_arr[i][j][k] = res
    elif len(img_arr.shape) == 2:
        for i in range(height):
            for j in range(width):
                img_arr[i][j] = (img_arr[i][j] > threshold) * 255
    
    pimg = pickle.dumps(img_arr)
    return xmlrpc.client.Binary(pimg)
server.register_function(send_back_binary_par, 'send_back_binary_par')

# Инверсия цвета
# На вход изображение RGB размерности (M, N, 3) со значениями 0-255
def send_back_inversion(bin_data):
    add_log('send_back_inversion')
    img_arr = pickle.loads(bin_data.data)
    
    height = img_arr.shape[0]
    width = img_arr.shape[1]   
    
    if len(img_arr.shape) == 3:
        for i in range(height):
            for j in range(width):
                for k in range(3):
                    img_arr[i][j][k] = 255 - img_arr[i][j][k]
    elif len(img_arr.shape) == 2:
        for i in range(height):
            for j in range(width):
                img_arr[i][j] = 255 - img_arr[i][j]
    pimg = pickle.dumps(img_arr)        
    return xmlrpc.client.Binary(pimg)
server.register_function(send_back_inversion, 'color_inversion')

# перевенуть картинку в вертикали
def send_back_flipped(bin_data):
    add_log('send_back_flipped')
    img_arr = pickle.loads(bin_data.data)

    height = img_arr.shape[0]
    width = img_arr.shape[1]
    
    if len(img_arr.shape) == 3:
            for i in range(height//2):
                img_arr[i], img_arr[height - i - 1] = img_arr[height - i - 1], img_arr[i]
    else:
        for i in range(height//2):
            img_arr[i], img_arr[height - i - 1] = img_arr[height - i - 1], img_arr[i]
    
    pimg = pickle.dumps(img_arr)       
    return xmlrpc.client.Binary(pimg)
server.register_function(send_back_flipped, 'send_back_flipped')

print("Listening on port 8008...")
server.serve_forever()



Listening on port 8008...
